In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
import numpy as np
from qiskit import *
%matplotlib inline
from math import pi, sqrt
import random

In [2]:
backend = Aer.get_backend('statevector_simulator')
def getBloch(quantumC):
    return visualization.plot_bloch_multivector(execute(quantumC, backend).
                                         result().get_statevector())

In [3]:
def stateGenerator(state):
    state = state[::-1]
    current = int(state, 2)
    newState = [0 for i in range(0, 2**(len(state)))]
    newState[current] = 1
    return newState

In [4]:
def getMeasure(quantumC, n):
    measures = []
    for i in range(0, n):
        outcome = execute(quantumC, backend).result()
        hist = outcome.get_counts()
        for i in hist.keys():
            measures.append(i)
    return measures

In [5]:
def makeALUCirc(n, op, inp):
    initState = op + inp
    print(initState)
    qr = QuantumRegister(n)
    qc = ClassicalRegister(n)
    circ = QuantumCircuit(qr, qc)
    initializedState = stateGenerator(initState)
    circ.initialize(initializedState, qr)
    return circ

In [6]:
opLUT = {'+': '11000',
          '-': '11100',
          'xor': '10000',
         'xnor': '10100',
         'nop': '00000',
         '+1': '11010',
         '-1': '11111',
         'neg': '00100',
        }
#Look Up Table - FPGA, Dictionary in Python

In [7]:
def performOp(inp1, inp2, op):
    circuit = makeALUCirc(7, opLUT[op], str(inp1) + str(inp2))
    circuit.csx(0, 6)
    circuit.csx(5, 6)
    circuit.cx(0, 5)
    circuit.crx(-pi/2, 5, 6)
    circuit.cx(0, 5)
    circuit.csx(1, 6)
    circuit.csx(3, 6)
    circuit.cx(1, 3)
    circuit.crx(-pi/2, 3, 6)
    circuit.cx(2, 6)
    circuit.cx(4, 6)
    circuit.measure(6, 6)
    return getMeasure(circuit, 1)[0][0]

In [27]:
performOp(1, 0, 'nop')

0000010


'0'